<div style="padding: 20px;
  text-align: center;
  background: #cae7ed;
  color: #003049;
  font-size: 14px;">
  <h2><b>Inclass: Introduction to Machine Learning 2</b></h2>
  <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/0*tVCene42rgUTNv9Q.png" alt="drawing" width="150"/>
   <hr>
</div>

- Part 6 of Data Analytics Specialization
- Course length: 9 Hours
- Instructor: Rany Dwi Cahyaningtyas

*Managed and developed by Algoritma Products Team Division*

In [1]:
# library preparation
import pandas as pd

# pandas output display setup
pd.options.display.float_format = '{:,.2f}'.format

# Introduction

## Time Series & Forecasting

- **Time Series** merupakan data yang berhubungan dengan waktu yang memiliki interval waktu yang sama (memiliki periode pengambilan data sama). Contoh:
    - Data harga saham
    - Data kenaikan harga cabai
    - Pendapatan bulanan
    - Data curah hujan

- **Forecasting** merupakan **proses prediksi** untuk data time series berdasarkan pola data historiknya.

## Regression vs Time Series

| **Karakteristik** | **Regresi**                              | **Peramalan/Forecasting**                      |
|-------------------|------------------------------------------|------------------------------------|
| **Tujuan Utama**  | Menemukan hubungan antara variabel dependen dan independen.       | Meramalkan nilai masa depan berdasarkan data historis.        |
| **Variabel**      | Terdapat variabel dependen (Y) dan variabel independen (X1, X2, ...)          | Mungkin hanya satu variabel dependen (misalnya, penjualan)        |
| **Fungsi**        | Memprediksi nilai Y berdasarkan nilai-nilai X.          | Memprediksi nilai masa depan Y berdasarkan data masa lalu.     |
| **Hubungan**      | Menunjukkan hubungan linear atau  non-linear antara variabel.       | Fokus pada tren, musiman, dan faktor lain yang memengaruhi.     |

# Working with Time Series Data

**Time Series Workflow**

1. [Import Data](#0)
2. [Data Wrangling](#1)
3. [Exploratory Data Analyis](#2) 
4. [Modeling](#3)
5. [Model Interpretation](#4)
6. [Forecast Evaluation](#6)
7. [Model Tuning](#6)

## 1. Import Data <a id="0" ></a>

📍 **Dataset** : Data time series penjualan harian dari perusahaan perangkat lunak terbesar di Rusia bernama **1C Company** dari [Kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview)
> - Data mencakup beberapa tahun dan berisi tentang informasi penjualan harian berbagai produk perangkat lunak di berbagai toko

📍 **Bussiness Problem** : Memprediksi total penjualan untuk setiap produk disuatu toko di bulan selanjutnya

🎯 **Tujuan**: Membantu mengoptimalkan manajemen inventaris dan strategi penjualan

Deskripsi data:
- Data terdiri dari 2,935,849 observasi baris dan 8 variabel atau kolom
- Glossary Data:
    + `date`: tanggal dengan format **dd.mm.yyyy**
    + `date_block_num`: urutan angka untuk bulan (Jan 2013 = 0, Feb 2013 = 1, ... Okt 2015 = 33)
    + `shop_id`: nomor identitas toko (0-59)
    + `ship_name`: nama toko
    + `item_id`: nomor identitas produk
    + `item_price`: harga barang di tanggal tersebut
    + `item_cnt_day`: jumlah barang terjual di tanggal tersebut
    + `total_revenue`: pendapatan kotor per hari (`item_cnt_day` * `item_price`)

## 2. Data Wrangling <a id="1" ></a>

🔎 Data time series merupakan data yang dikumpulkan pada interval **waktu yang teratur**. Dalam kasus ini kita ingin menganalisis tentang **total penjualan** perangkat lunak yang dikumpulkan **setiap hari**.

Beberapa hal yang wajib diperhatikan ketika melakukan analisis data time series yaitu:

- Konversi tipe kolom **date** ke `datetime64[ns]`
- Memastikan data terurut dengan baik (terlampau >> terbaru)
- Membuat data menjadi interval yang tetap (harian)

### Konversi tipe kolom **date** ke `datetime64[ns]`

Kolom yang akan masuk ke dalam pemodelan `prophet` harus bertipe data `datetime64[ns]`, oleh sebab itu wajib kita lakukan perubahan tipe data terlebih dahulu.

In [ ]:
# cek tipe data sebelum konversi


Sebelum melakukan konversi tipe data, kita perlu memeriksa cara penulisan tanggal pada data kita terlebih dahulu, agar tidak terjadi kesalahan pada saat konversi (misal, bulan menjadi tanggal, atau sebaliknya).

In [ ]:
# cek cara penulisan tanggal
sales.head(2)

> Format: ____

Jika kita sudah yakin, maka kita dapat melakukan konversi tipe data ke `datetime64[ns]`

In [ ]:
# to datetime
sales['date'] = 

In [ ]:
# cek tipe data setelah konversi
sales.dtypes

### Memastikan data terurut dengan baik

Salah satu syarat dalam membuat model Time Series adalah data tanggal harus terurut dengan baik dari data **paling lampau** sampai dengan data **paling baru**.

Untuk mengurutkan data, kita dapat menggunakan method `.sort_values()`

In [ ]:
# Sort the data ascending by date column
sales = 

Setelah tanggal sudah terurut, kita dapat melihat range tanggal pada data kita.
- Penjualan paling awal: ___
- Penjualan paling akhir: ___

### Membuat data menjadi interval yang tetap

Hal yang penting dari sebuah data time series adalah memiliki **interval waktu yang tetap**, seperti interval harian, mingguan ataupun bulanan. Tidak boleh ada data yang **berulang pada tanggal yang sama**.

Pada data di atas, terdapat beberapa observasi dalam satu hari untuk toko yang sama, karena satu toko dapat menjual beberapa produk *software* yang berbeda dalam satu hari. Oleh karena itu, data kita masih dalam interval yang tidak tetap. 

🎯 Kita akan melakukan agregasi untuk menghitung jumlah penjualan (`item_cnt_day`) dan total pendapatan (`total_revenue`) untuk tiap tanggal `date` di masing-masing toko `shop_id`.

❗️ Ubah data tersebut menjadi data yang diinginkan


|     date    | shop_id | total_qty | total_revenue |
|:-----------:|:-------:|:---------:|:-------------:|
|  2013-01-01 |   2    |   24.00   | 31,886.00      |
|  2013-01-01 |   7    |   89.00   | 59,993.00      |
|  2013-01-01 |   8    |   50.00   | 44,516.00      |
|  2013-01-01 |   13   |   31.00   | 6,043.00       |
|  2013-01-01 |   14   |   66.00   | 48,847.00      |

In [ ]:
# menghitung jumlah item & revenue per hari untuk tiap toko
daily_sales = sales.groupby(['date', 'shop_id']).sum()[['item_cnt_day', 'total_revenue']].reset_index()

# mengubah nama kolom
daily_sales = daily_sales.rename(columns = {'item_cnt_day': 'total_qty'})

daily_sales.head()

Tiap toko akan memiliki data dengan tanggal yang berurutan dan tidak berulang. Untuk memastikan agregasi kita sudah benar, pastikan untuk `shop_id == 2`

In [ ]:
daily_sales[daily_sales['shop_id'] == 2]

## 3. Exploratory Data Analysis <a id="2"></a>

**❓ Task 1** Sebagai sebuah perusahaan software terbesar di Rusia, 1C Company menjual produknya ke berbagai toko. Ada berapa jumlah toko yang menjual software buatan 1C?

In [ ]:
# code here


📝 **Insight**: ___

**❓ Task 2** Dari toko-toko tersebut, manakah Top 3 `shop_id` dengan `total_qty` terbanyak?

In [ ]:
# code here
daily_sales[[___, ___]].groupby(___).sum()

In [ ]:
# sort_values()


📝 **Insight**: Dari hasil analisis kita di atas, terlihat **toko ___, ___, dan ___** adalah 3 toko dengan penjualan produk terbanyak.

**❓ Task 3** Kita akan berfokus kepada 3 toko ini saja. Buat dataframe baru bernama `top_3` yang berisi 3 toko tadi saja!

In [ ]:
# membuat syarat untuk conditional subseting
conditions = (daily_sales['shop_id'] == ___) | (daily_sales['shop_id'] == ___) | (daily_sales['shop_id'] == ___)

# filtering
top_3 = daily_sales[conditions]
top_3

### Visualization: Multiple vs Multivariate Time Series

Sebelum kita masuk ke tahapan pembuatan model, kita perlu melakukan analisis berdasarkan visualisasi datanya. Pada bagian ini kita akan menggunakan library **`matplotlib` dan `seaborn`**.

📌 Informasi tambahan tentang perbedaan *multiple* dengan *multivariate* pada kasus time series, berikut adalah definisi dari keduanya:

| **Karakteristik**                   | **Data Time Series Multiple**                 | **Data Time Series Multivariate**         |
|-------------------------------------|----------------------------------------------|------------------------------------------|
| **Definisi**                        | Terdapat **satu variabel** dari **banyak objek** yang ditinjau dari waktu ke waktu.       | Terdapat **banyak variabel** dari hanya **satu objek** yang ditinjau dari waktu ke waktu |
| **Tujuan**                          | Membandingkan pola dan tren beberapa object dari seri waktu. | Memahami hubungan antara variabel-variabel yang berubah seiring waktu dan mengidentifikasi interaksi antara variabel-variabel tersebut. |

In [ ]:
# data visualization library
import matplotlib.pyplot as plt
import seaborn as sns

#### Multiple Time Series

Pada kasus kita, multiple time series adalah ketika kita meninjau fluktuasi dari variabel `total_qty` pada **tiga toko** terpopuler.

In [ ]:
multiple = top_3.pivot_table(index='date', # Kolom date
                                   columns='shop_id', # Dipisah berdasarkan kode toko
                                   values='total_qty', # Total produk terjual
                                   aggfunc='sum')
multiple.head()

In [ ]:
multiple.plot(subplots=True, figsize=(10, 5));


📈 Dengan melihat visualisasi tersebut kita mendapatkan:

**1. Bagaimana pola penjualannya?**
> ......

**2. Bagaimana tren tiap toko?**
> ......

**3. Bagaimana fluktuasinya?**
> ......


#### Multivariate Time Series

Pada kasus kita, multivariate time series adalah ketika kita meninjau pergerakan dari `total_qty` dan `total_revenue` dari toko 31 saja. Kita akan melakukan conditional subsetting dari data `daily_sales` untuk membuat `daily_sales_31`.

In [ ]:
daily_sales_31 = top_3[top_3['shop_id'] == 31].reset_index(drop=True)

In [ ]:
daily_sales_31.set_index('date')[['total_qty', 'total_revenue']].plot(subplots=True,figsize=(10, 5))
plt.suptitle('DAILY SOFTWARE SALES: STORE 31')
plt.show()

📈 Dengan melihat visualisasi tersebut kita mendapatkan:

**1. Bagaimana fluktuasinya?**
>  ......

**2. Bagaimana hubungan antar variabelnya??**
> ......



## 4. Modelling using [`prophet`](https://facebook.github.io/prophet/) <a id="3"></a>

Konsep dasar dalam pemahaman kasus time series adalah dengan melakukan pemecahan komponen dari data time series (***decompose***). *Decompose* time series memiliki banyak macam dan algoritma yang dapat digunakan. Konsep yang digunakan oleh `Prophet` dalam melakukan decompose adalah metode **Generative Additive Model (GAM)**

**General Additive Model (GAM)**

> Salah satu kemajuan terbaru dalam pemodelan data time series yang memandang time series sebagai hasil penjumlahan dari komponennya: 
>> $Y(t) = T(t) + S(t) + E(t)$
> - **Trend (T)**: Pergerakan jangka panjang dalam nilai rata-ratanya.
> - **Seasonality (S)**: Efek musiman berulang.
> - **Residuals (E)**: Komponen tak teratur atau fluktuasi acak yang tidak dijelaskan oleh tren dan musiman.

**`prophet`**
> - Dikembangkan oleh facebook untuk membuat pemodelan data time series lebih mudah diakses oleh non ekspert.
> - Pendekatan yang sederhana sehingga mudah dipahami dan cocok untuk pemula
> - `prophet` memanfaatkan GAM yang menambahkan efek hari libur untuk memodelkan jadwal yang tidak teratur.

<div class="alert">

📍 **Business Problem** :  Melakukan prediksi penjualan harian untuk toko 31.

📍 **Tujuan**: Membantu mengoptimalkan manajemen inventaris dan strategi penjualan.

📍 **Data**: Kita akan menggunakan data penjualan harian untuk toko 31. Toko ini cocok digunakan karena memiliki :

1. Relevansi bisnis: sebagai toko dengan penjualan tertinggi.
2. Sederhana dan terfokus: Meminimalkan kompleksitas model dan memudahkan pengembangan strategi forecasting yang dapat diimplementasikan dengan mudah.
3. Ketersediaan data: toko 31 memiliki data yang lebih lengkap dan konsisten per hari dibandingkan toko lain.

Catatan: Perlu diingat bahwa pemilihan toko ini dapat disesuaikan jika terdapat kebutuhan bisnis spesifik yang memerlukan analisis pada toko lain.

</div>

### **Data Preparation**

Untuk menggunakan `prophet`, kita perlu mempersiapkan data time series ke dalam format yang spesifik sesuai kebutuhan library ini. 

<div class="alert">

- Data frame harus terdiri dari 2 kolom dengan nama sebagai berikut:
    + `ds`: kolom penanda waktu, harus bertipe `datetime64`
    + `y`: nilai yang ingin diprediksi

</div>

🔻 Pada contoh di bawah kita akan menggunakan `total_qty` pada toko 31 sebagai nilai yang ingin kita forecast.

In [ ]:
# hanya mengambil kolom date & total_qty
daily_total_qty = daily_sales_31[['date', 'total_qty']]

# mengganti nama kolom menjadi ds & y
daily_total_qty = daily_total_qty.rename(
    columns = {'date' : 'ds',
               'total_qty' : 'y'})

daily_total_qty.head()

### **Fitting Model**

🔻 Setelah semua persiapan data selesai. Kini kita tugaskan `prophet` untuk mencari pola dari data time series kita. Adapun langkah pembuatan model, yaitu:

![](assets/fit-model.jpg)

In [ ]:
# load library
from prophet import Prophet

🔻Inisiasi object model menggunakan fungsi `Prophet()`

🔻 Fitting data `daily_total_qty` menggunakan method `fit()`

In [1]:
# melakukan fitting / training


### **Forecast Model**

Berikut untuk tahapan pada saat melakukan forecasting model Prophet:

![](assets/forecast-model.jpg)


🔻 **Menentukan Periode Forecasting**

> Dalam kasus ini, kita melakukan forecasting selama **1 tahun kedepan**

❗️ Membuat dataframe masa depan menggunakan method `make_future_dataframe()` dari objek model kita yaitu `model_31` dan tentukan parameter:

- `periods` = banyaknya data yang akan kita forecast
- `freq` = format data/frequency. `D` (day), `M` (month), dll

In [ ]:
# dataframe baru berisikan data waktu lama disertai data waktu baru
future_31 = ___                                        

🔻 Prediksi data

Kita akan menggunakan dataframe `future_31` ini untuk melakukan forecasting menggunakan method `.predict()`.

In [ ]:
forecast_31 = ___
forecast_31.head()

Dari berbagai nilai di atas kita akan mengambil beberapa informasi saja, yaitu:
- `ds`: Tanggal
- `trend`: Nilai trend
- `weekly`: Nilai seasonal mingguan
- `yearly`: Nilai seasonal tahunan
- `yhat`: Nilai prediksi

<details>
    <summary>🔻 Detail deskripsi setiap kolom hasil predict </summary>
    
- `yhat_lower`: batas bawah dari nilai prediksi
- `yhat_upper`: batas atas dari nilai prediksi

- `trend_lower`: batas bawah dari tren
- `trend_upper`: batas atas dari tren

- `weekly_lower`: batas bawah dari komponen mingguan dari nilai prediksi
- `weekly_upper`: batas atas dari komponen mingguan dari nilai prediksi


- `yearly_lower`: batas bawah dari komponen tahunan dari nilai prediksi
- `yearly_upper`: batas atas dari komponen tahunan dari nilai prediksi

- `additive_terms`: komponen tambahan dari nilai prediksi
- `additive_terms_lower`: batas bawah dari komponen tambahan dari nilai prediksi
- `additive_terms_upper`: batas atas dari komponen tambahan dari nilai prediksi

- `multiplicative_terms`: komponen perkalian dari nilai prediksi
- `multiplicative_terms_lower`: batas bawah dari komponen perkalian dari nilai prediksi
- `multiplicative_terms_upper`: batas atas dari komponen perkalian dari nilai prediksi
    
</details>

In [ ]:
forecast_31[['ds', 'trend', 'weekly', 'yearly', 'yhat']]

<div class="alert alert-danger"><center>
  <details>
      <summary><b>💬 Ingat Kembali: Konsep GAM </b></summary>GAM merupakan model yang bersifat aditif, yang berarti <br> menjelaskan data time series sebagai hasil penjumlahan dari beberapa komponen.
</div> 

💬 Pada kasus data kita, model kita telah mengekstrak 3 tipe komponen: `trend`, `weekly` seasonaliti, dan `yearly` seasonaliti. Oleh sebab itu, formulanya menjadi

$yhat(t) = T(t) + S_{weekly}(t) + S_{yearly}(t)$

❗️ Pembuktian bahwa `forecast_31` memuat fungsi berikut `yhat` = `trend` + `weekly` + `yearly`.

In [ ]:
# penjumlahan manual
(forecast_31['trend'] + forecast_31['weekly'] + forecast_31['yearly']).head()

In [ ]:
# sama dengan nilai yhat
(forecast_31['yhat']).head()

### **Visualisasi**

Setelah membuat model dan melakukan forecast, kita akan melakukan visualisasi dari `model_31` dengan method `.plot()` yang diisi dengan hasil forecast-nya (`forecast_31`). 

Hasil plot akan menampilkan data asli berupa titik-titik hitam dan hasil model fitted serta forecast dengan garis biru.

In [ ]:
# your code here


## 5. Model Interpretation <a id="4" ></a>

Model time series yang kita buat selain untuk dapat digunakan untuk meramalkan kejadian di masa depan juga dapat digunakan sebagai penarikan insight terhadap data historis.

Dalam melakukan interpretasi, kita dapat melihat komponen time series dengan menggunakan fungsi `.plot_components()` dari objek `model_31`

In [ ]:
# your code here


🔻 Komponen:

- **Trend:** Pergerakan jangka panjang dari data kita.

- **Seasonal:** Pola berulang dalam periode tertentu (musiman)
    - weekly: pola berulang mingguan
    - yeayly: pola berulang tahunan

❓ Insight apa saja yang dapat kita ambil dari visualisasi di atas?

- **TREND COMPONENT**
    - trendnya .... dari waktu ke waktu

- **WEEKLY SEASONALITY**
    - penjualan cenderung lebih tinggi di waktu ....
    - Penjualan terendah di hari ....

- **YEARLY SEASONALITY**
    - ....

## 6. Time Series Component

| **Komponen** | **Penerapan Bisnis**                                                                                                                                                 |
|--------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Trend**        | Bisnis dapat merencanakan pertumbuhan jangka panjang  dan mengidentifikasi perubahan pasar                                                                           |
| **Seasonality**  | Dengan memahami musiman, bisnis dapat menyesuaikan strategi pemasaran,  persediaan, dan penawaran khusus untuk mengikuti pola musiman.                               |
| **Holiday**      | Dengan memahami efek liburan, bisnis dapat merencanakan  persiapan khusus untuk menghadapi lonjakan permintaan atau  menawarkan promosi yang relevan selama liburan. |

### Trend Component

Trend adalah pergerakan data secara general dalam jangka panjang. `Prophet` menggunakan **linear regression** untuk mendapatkan trend yang dapat digambarkan sebagai berikut:

![](assets/trend.png)

📖 Further reading for [math behind trend in `prophet`](https://github.com/ranydwi/prophet_trend/blob/main/math_behind_trend.ipynb)

Dari visualisasi di atas kita dapat melihat komponen trend data penjualan dari waktu ke waktu untuk shop_id 31. Kita dapat mengamati apakah trend tersebut meningkat, menurun, atau tetap.

#### Automatic Changepoint Detection

<div class="alert alert-success">
Kelebihan dari `Prophet` yaitu dengan adanya <b>changepoint detection</b> yang bisa mendeteksi otomatis adanya perubahan signifikan pada trend. 
</div> 

`Prophet` akan membagi data kita pada beberapa titik dan menghitung trend per bagian secara terpisah.

🔎 **Default Setting**
> - `n_changepoints=25`: 25 jumlah changepoint
> - `changepoint_range=0.8`: 80% data awal tempat changepoints berada

> Ini berarti Prophet akan mencari hingga 25 changepoints yang mungkin dalam 80% awal data untuk membantu mengidentifikasi di mana tren mungkin berubah.

Changepoint dapat divisualisasikan dengan menggunakan fungsi `add_changepoints_to_plot()`.

In [ ]:
# for illustration purposes only, threshold = 0
from prophet.plot import add_changepoints_to_plot

fig = model_31.plot(forecast_31)
a = add_changepoints_to_plot(
    ax = fig.gca(), # plot
    m = model_31, # model
    fcst = forecast_31,
    threshold = 0) # hasil forecast

Dari 25 changepoints prophet akan menghitung besarnya tingkat perubahan pada slope dan akan menentukan mana yang paling signifikan. 

🔻 Model ini mendeteksi 3 changepoints yang signifikan

In [ ]:
fig = model_31.plot(forecast_31)
a = add_changepoints_to_plot(fig.gca(), model_31, forecast_31)

#### Adjusting Trend Flexibility

Jika trend dirasa terlalu fleksibel (overfit) atau kaku (underfit), Prophet menyediakan hyperparameter untuk merubah fleksibilatas dari trend, yaitu:

💬 **Parameter yang direkomendasikan untuk ditune**

- `changepoint_prior_scale` (default = 0.05): Fleksibilitas dari trend. Semakin besar nilainya, maka trend akan semakin fleksibel, begitupun pula sebaliknya Rekomendasi: range[0.001, 0.5]

1️⃣ Fitting Model

❗️Mengatur parameter `changepoint_prior_scale = 0.5` akan membuat trend lebih fleksibel dalam mendeteksi titik perubahan trend. Membantu menangkap pola yang lebih kompleks dalam data, namun juga meningkatkan resiko overfitting

In [ ]:
# membuat object trend tuning
model_tuning_trend = ___

# fitting model
model_tuning_trend.fit(daily_total_qty)

2️⃣ Forecasting

In [ ]:
# forecasting
future = model_tuning_trend.make_future_dataframe(periods = 365, freq = 'D')
forecast_trend_tuning = model_tuning_trend.predict(future)

3️⃣ Visualization

In [ ]:
# visualize
fig = model_tuning_trend.plot(forecast_tuning)
a = add_changepoints_to_plot(fig.gca(), model_tuning_trend, forecast_tuning)

📈 Dengan melihat visualisasi tersebut kita mendapatkan:

**Bagaimana pengaruh pada hasil visualisasi?**
> ......


### Seasonality Component

Selanjutnya kita akan membahas komponen time series lain yaitu seasonal. Seasonality adalah pola berulang untuk periode waktu tertentu. Seperti pola berulang untuk periode mingguan, bulanan atau tahunan. Berikut kita tinjau kembali plot masing-masing komponen pada data di toko 31.

In [ ]:
# lihat kembali plot komponen
fig = model_31.plot_components(forecast_31)

<div class="alert alert-success">
Kelebihan Prophet secara default, akan menentukan kondisi seasonal berdasarkan periodikal data yang disediakan. Serta kemudahan membentuk <b>multiple seasonality</b>
</div> 

Pada kasus kita, data tersusun dari Januari 2013 sampai akhir 2015 Quarter 3.

- Setiap data dengan periode harian akan memiliki **weekly seasonality**.
- Jika terdapat data > 2 tahun observasi harian, model akan memiliki **yearly seasonality**.
- Terdapat komponen seasonal lain, yaitu **daily seasonality** yang berusaha mencari pola tiap jam dalam 1 hari. Karena data kita tidak memiliki informasi jam, maka model kita tidak memiliki daily seasonality.

#### Fourier Order

Prophet menggunakan deret Fourier untuk memperkirakan efek seasonal.

💡 Fourier Order adalah parameter yang menentukan seberapa fluktuatif seasonality kita. Dengan meningkatkan order, kita dapat membuat seasonality yang dihasilkan menjadi lebih fleksibel (dan mungkin menyebabkan model menjadi overfit), begitu pula sebaliknya.

Berikut gambaran interaktif tentang Fourier: http://www.jezzamon.com/fourier/

![](assets/fourier.svg)

Default Fourier Order pada Seasonality Prophet:
- `weekly_seasonality = 3`
- `yearly_seasonality = 10`

[Reference Fourier Order](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#fourier-order-for-seasonalities)

In [ ]:
# create object
model_tuning_seasonality = Prophet(
    weekly_seasonality = 3, # default 3
    yearly_seasonality = ___) # defaultnya 10, kita tuning jadi 100

In [ ]:
# model fitting
model_tuning_seasonality.fit(daily_total_qty)

In [ ]:
# forecasting
## make future dataset
future = model_tuning_seasonality.make_future_dataframe(periods=365, freq='D')
## predict
forecast = model_tuning_seasonality.predict(future)

In [ ]:
# visualize
fig = model_tuning_seasonality.plot_components(forecast)

📈 Dengan melihat visualisasi tersebut kita mendapatkan:

**Bagaimana pengaruh pada hasil visualisasi?**
> ...


#### Custom Seasonality

❓ Sebagai seorang data analyst, anda menemukan suatu kasus bisnis seperti berikut:
> Penjualan di bisnis Anda sangat dipengaruhi oleh tanggal bulanan. Kebanyakan dari pembeli cenderung membeli barang berdasarkan waktu pembayaran gaji mereka. Karena hal ini tidak ditangkap pada kondisi seasonal weekly dan yearly, kita perlu mendefinisikan seasonal custom. 

Custom seasonality dapat dilakukan dengan cara:

- **Menambahkan seasonality tambahan** dapat menggunakan method `.add_seasonality()` setelah membuat object Prophet dan sebelum melakukan fitting.

  Pada fungsi `.add_seasonality()`, terdapat 3 parameter yang dapat kita gunakan, yaitu:
  - `name`: Nama seasonality
  - `period`: Pola seasonal yang ingin dianalisa
  - `fourier_order`: Sebarapa cepat seasonality berubah

- **Menghilangkan efek seasonal default** pada fungsi `Prophet()` jika dibutuhkan (contoh: `yearly_seasonality = False`).

💡 Nilai Fourier order yang direkomendasikan berdasarkan seasonalnya:
- weekly seasonality = 3
- monthly seasonality = 5
- yearly seasonality = 10

**🔻 Task**

Buatlah monthly seasonality dan coba tiadakan efek yearly seasonality.
Sehingga formula yang dibentuk menjadi:

$yhat(t) = T(t) + S_{weekly}(t) + S_{monthly}(t)$

1️⃣ Menghapus yearly seasonality

In [ ]:
# fitting model remove seasonality
model_custom_seasonality = ___

2️⃣ Menambahkan Custom Seasonality

- Montly Seasonality
- `period = 30.5` (rata-rata jumlah hari dalam satu bulan)
- `fourier_order=5`
    > - biasanya [3,5] untuk model yang sederhana
    > - mengontrol seberapa kompleks model seasonality yang digunakan. Semakin tinggi semakin fleksibel modelnya (biasanya `fourier_order=10`
    

In [ ]:
## add seasonality


## fitting model


3️⃣ Forecasting & visualization

In [ ]:
# forecasting
## make_future_dataframe
future = 

## predict
forecast_custom_season = 

# visualize
fig = model_custom_seasonality.plot_components(forecast_custom_season)

📈 Dengan melihat visualisasi tersebut kita mendapatkan:

**Bagaimana pengaruh pada hasil visualisasi?**
> ...

### Holiday Effects

Prophet dapat memperhitungkan *holiday effect* ke dalam model time series. *Holiday effect* adalah kejadian tidak biasa yang mungkin berdampak signifikan pada rangkaian waktu, seperti **hari libur nasional** atau **promosi penjualan**.

Berbeda dengan seasonality yang dapat terdeteksi secara otomatis, *holiday effect* perlu **ditentukan secara manual** oleh pengguna. Dengan memasukkan *holiday effect* ke dalam model, Prophet dapat menangkap dampak peristiwa penting ini dan memberikan forecast yang lebih baik.

🔻 Sekarang kita akan melihat data kita dari model pertama yang kita buat. Dari visualisasi kita dapat melihat adanya penjualan yang signifikan di setiap akhir tahun (mencapai lebih dari 800 penjualan per hari).

In [ ]:
# for illustration purposes only
fig = model_31.plot(forecast_31)
plt.axhline(y=800, color='red', ls='--')
plt.show()

Mari kita lihat penjualan dengan jumlah di atas 800 menggunakan conditional subsetting.

In [ ]:
# conditional subseting y > 800 dari data daily_total_qty
daily_total_qty[daily_total_qty['y'] > 800]

Ternyata penjualan yang sangat tinggi terjadi antara tanggal **27-31 Desember** pada 2 tahun data kita. Dengan ini kita dapat mengasumsikan adanya fenomena tahun baru yaitu saat dimana orang-orang menggunakan bonus akhir tahun mereka untuk berbelanja.

Dari sini kita dapat mempersiapkan sebuah dataframe baru yang memiliki informasi data holiday yang akan kita atur.

Kolom-kolom yang harus dipersiapkan yaitu:
- `holiday` : nama hari libur yang nilainya unik
- `ds` : informasi waktu
- `lower_window` : jumlah periode waktu **sebelum** tanggal hari libur yang diasumsikan memiliki pengaruh (<=0)
- `upper_window` : jumlah periode waktu **setelah** tanggal hari libur yang diasumsikan memiliki pengaruh (>=0)

⚠️ Informasi ini harus mencakup seluruh kejadian yang akan kita gunakan, baik pada model fitting maupun forecast (mundur sejauh historikal data kita, dan maju sejauh kita ingin melakukan prediksi)

In [ ]:
new_year = pd.DataFrame({
    'holiday' : 'new_year',
    'ds' : pd.to_datetime(['2013-12-31', '2014-12-31', # tanggal historikal
                           '2015-12-31']), # future date
    'lower_window' : -4, # terhitung 27-30 des
    'upper_window' : 0
})
new_year

Setelah dataframe `hari_libur` dipersiapkan, kita perlu memasukkan informasi ini ke dalam `Prophet()` di parameter `holidays`.

In [ ]:
# model kosong
model_holiday = ___

# model fitting
model_holiday.fit(daily_total_qty)

# forecasting
future = model_holiday.make_future_dataframe(periods = 365, freq = "D")
forecast = model_holiday.predict(future)

# visualize
fig = model_holiday.plot(forecast)

Dengan adanya informasi hari libur akhir tahun, terlihat bahwa model yang dihasilkan lebih baik menangkap pola di akhir tahun dibandingkan hanya menggunakan efek yearly seasonality. Jika kita plot komponennya, kita dapat melihat adanya efek komponen holiday yang terlampir:

In [ ]:
fig = model_holiday.plot_components(forecast)

# Time Series Workflow 

![workflow](assets/prophet-workflow.jpg)

## Implemented - Time Series Workflow

### Data Preparation

Yang perlu diingat, Model Prophet mewajibkan membuat data ke dalam 2 kolom (**ds** dan **y**)

[Link to section](#Data-Preparation)

In [ ]:
daily_total_qty.head()

### Time Series Cross Validation

Seperti di kasus klasifikasi/regresi dimana dilakukan evaluasi untuk mengetahui sebaik apa performa model yang telah dibangun. Pada model time series, terdapat juga tahapan evaluasi model dimana kita akan membagi data kita menjadi **data train** dan **data test**

💬 Evaluasi model forecasting dengan metode splitting melibatkan pembagian data menjadi dua bagian:
> - **Data Latih (Train Data)**: Digunakan untuk melatih model time series untuk mengidentifikasi pola-pola dasar seperti tren dan seasonal
> - **Data Uji (Test Data)**: berfungsi sebagai **unseen data** untuk melihat performa model

Konsep CV untuk model TS akan sama dengan model klasifikasi maupun model regresi, dimana *data trainnya akan lebih banyak di bandingkan data test*. Akan tetapi terdapat satu kriteria lainnya yang harus dipenuhi yaitu, 
- **Data Train** akan menggunakan **data awal** 
- **Data Test** akan menggunakan **data akhir**

![](assets/cv.png)

Mari kita lihat kembali range data kita

In [ ]:
daily_total_qty['ds'].describe()

Pada data ini, kita memiliki range waktu dari Januari 2013 sampai Q3 tahun 2015. Misalkan kita ingin menggunakan data di tahun 2015 sebagai test data, sisanya untuk train data. Kita dapat memisahkan data kita dengan conditional subseting:

In [ ]:
train = ___
test = ___

print(f'Train size: {train.shape}')
print(f'Test size: {test.shape}')

<div class="alert alert-danger">
Sekarang mari kita latih model kita menggunakan data tahun 2013-2014 dan forecast untuk tahun 2015 (<b>303 hari</b>).

<hr>

🔻 **Task**

Buat model dengan spesifikasi berikut!
- Atur `holidays` = dataframe holiday yang sudah dibuat
- `yearly_seasonality` = True
- tambahkan seasonality = `monthly` dengan `period = 30.5` dan `fourier_order = 5`
</div> 

### Fitting Model 

In [ ]:
# add holiday & yearly seasonality
model_final = Prophet(...)

# add seasonality monthly
model_final.add_seasonality(name = 'monthly',
                            period = ___,
                            fourier_order = ___)

📍 Fitting model hanya untuk **train** data

In [ ]:
# fitting model
model_final.fit(___)

### Forecast Model

📍 Banyaknya periode forecast = sebanyak data test

In [ ]:
future_final = 
forecast_final = 

### Model Interpretation - Visualize

In [ ]:
# visualize
fig = model_final.plot(forecast_final)
plt.scatter(x=test['ds'], y=test['y'], s=10, color='red')
plt.show()

## 7. Evaluation Metrics

Berdasarkan plot di atas model kita mampu untuk meramalkan kondisi di masa depan secara general, tetapi pada beberapa bagian masih berbeda antara data asli dengan data prediksi. **Perbedaan antara nilai asli dengan prediksi** disebut dengan **error** dan kita harus menghitung nilainya untuk mendapatkan kesimpulan evaluasi model. Berikut ini adalah beberapa metrics yang dapat digunakan:

- Mean Absolute Error (MAE)

    Rata-rata seberapa menyimpang hasil *forecast* terhadap data aktual.

$$
MAE = \frac{\sum_{i=1}^{n}\left | y_{pred, i} - y_{real, i} \right |}{n}
$$

> 🔎 **Interpretasi MAE**: MAE mengukur kesalahan rata-rata dalam satuan yang sama dengan data aktual. Misalnya, jika MAE adalah 10, ini berarti prediksi rata-rata selisih sebesar 10 unit dari data aktual.

- Mean Precentage Absolute Error

    Sama seperti MAE, namun dalam bentuk persentase, sehingga lebih mudah diinterpretasikan, namun tidak dapat digunakan ketika ada nilai 0 pada data kita
$$
MAPE = \frac{1}{n}\Sigma{\frac{| y-\hat y|}{y}} \times 100\%
$$  

> 🔎 **Interpretasi MAPE**:MAPE mengukur kesalahan rata-rata dalam bentuk persentase dari data aktual. Misalnya, jika MAPE adalah 5%, ini berarti prediksi rata-rata memiliki kesalahan sebesar 5% dari data aktual.


Pada course ini kita akan menggunakan MAPE karena paling mudah diinterpretasikan, dengan catatan tidak ada nilai 0 pada data kita.

Untuk menghitung MAPE kita dapat mengugnakan fungsi `mean_absolute_percentage_error` dari `scikit-learn`.

💡 Nilai aktual adalah `y` pada data awal dan nilai prediksi adalah `yhat` pada data forecast

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# hasil prediksi train
forecast_train = 

# nilai MAPE train
train_mape = mean_absolute_percentage_error(y_true = ...,
                                            y_pred = ...)
train_mape                                           

In [ ]:
# hasil prediksi test
forecast_test = 

# nilai MAPE test
test_mape = mean_absolute_percentage_error(y_true = ...,
                                           y_pred = ...)
test_mape

## 8. Tuning Model - Hyperparameter Tuning <a id="6" ></a>

*Hyperparameter tuning* adalah langkah penting dalam membangun model machine learning untuk mencapai performa terbaik.

🎯 **Tujuan**: 
> Menemukan **kombinasi hyperparameter terbaik** yang **meminimalkan error**

💬 **Metode Hyperpatameter Tuning**
> - Menggunakan **Algoritma Grid Search**: metode *brute-force* yang membangun model untuk setiap kombinasi hyperparameter yang ditentukan dan kemudian mengevaluasinya.
> - Metode ini bisa memakan waktu dan sumber daya komputasi, tetapi metode ini menjamin menemukan hyperparameter optimal yang sesuai dengan rentang nilai yang ditentukan.

💬 **Langkah-langkah menggunakan Algoritma Grid Search untuk mencari hyperparameter terbaik**
> 1.  Menentukan nilai-nilai dari hyperparameter yang ingin dicoba
> 2.  Melakukan iterasi pembuatan model dan menghitung nilai MAPE-nya menggunakan *for-loop*
> 3.  Mendapatkan kombinasi hyperparameter optimal yang memiliki MAPE terkecil

💬 **Referensi**
> - Silakan buka [link ini](https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning) untuk melihat rekomendasi hyperparameter yang dapat di-tuning pada model Prophet.<br>

🔻 Sebagai seorang data analyst, kita akan mencoba beberapa kemungkinan hyperparameter. Beberapa diantaranya yaitu:
- `changepoint_prior_scale` (range 0.001 - 0.5): mengatur trend, semakin besar semakin fluktuatif 

1️⃣ Tentukan hyperparameter dan masukkan beberapa nilai didalamnya. Serta buat variabel untuk mengisi kombinasi hyperparameter dan nilai MAPE

In [ ]:
from sklearn.model_selection import ParameterGrid

# Grid search parameters
param_grid = {
    'changepoint_prior_scale' : [0.005, 0.01, 0.05, 0.5],
    'yearly_seasonality' : [10,20]
}

# generate all combinations of parameters
all_params = list(ParameterGrid(param_grid))
all_params

2️⃣ Melakukan iterasi dengan memasukkan kedalam pembuatan model Prophet - Fitting Model - Evaluasi

In [ ]:
mape = [] # store the result for each params here

# iterate each parameter
for parameter in all_params:
    # print parameter
    print(parameter)
    # hyperparameter tuning
    model = Prophet(
        changepoint_prior_scale = parameter['changepoint_prior_scale'],
        yearly_seasonality= parameter['yearly_seasonality'],
        holidays = new_year
    )

    # train model with data train
    model.fit(train)

    # forecasting
    future = model.make_future_dataframe(periods=303, freq='D') # 303 days (test size)
    forecast = model.predict(future)
    
    # Evaluation metrics: MAPE
    forecast_test = forecast[forecast['ds'] >= '2015-01-01']
    test_mape = mean_absolute_percentage_error(y_true=test['y'],
                                               y_pred=forecast_test['yhat'])
    
    mape.append(round(test_mape, 2))

🔻 Dari variabel `all_params` dan `mape` kita dapat menyusun dataframe yang menyimpan kedua informasinya lalu mengurutkannya dari yang errornya paling kecil.

In [ ]:
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape

tuning_results.sort_values(by = 'mape')

3️⃣ Dengan informasi parameter terbaik ini, kita lakukan kembali fitting model yang akan digunakan untuk forecasting.

In [ ]:
import numpy as np

# mengambil hyperparameter dengan mape terendah
best_params = all_params[np.argmin(mape)]
best_params

4️⃣ **Buat ulang model** -> Dengan informasi parameter terbaik ini, kita lakukan kembali fitting model yang akan digunakan untuk forecasting.

In [ ]:
# model kosong
model_best = Prophet(...)
# model fitting
model_best.fit(train)

# forecasting
future_best = model_best.make_future_dataframe(periods=365, freq='D')
forecast_best = model_best.predict(future_best)

# visualize
fig = model_best.plot(forecast_best)

# References

- [Prophet Documentation](https://facebook.github.io/prophet/docs/quick_start.html)
- [Paper: Forecasting at Scale](https://peerj.com/preprints/3190/)
- [Prophet Hyperparameter Tuning](https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning)
- [Algoritma: Time Series Forecasting using `prophet` in R](https://business-forecasting.netlify.app/#5_time_series_forecasting_using_prophet)
- [Algoritma: Time Series Forecasting using `prophet` in Python](https://github.com/tomytjandra/tsf-prophet)